In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

2022-03-22 12:22:32.956796: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-22 12:22:32.956815: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:


def create_df(journey):
    comunio = pd.read_csv(f'../data/pruebas/only_comunio_stats_J{journey}.csv')
    clas = pd.read_excel(f'../data/classification_J_{journey}.xlsx', sheet_name=f'classification_J_{journey}', index_col='Unnamed: 0')
    cal = pd.read_csv('../data/pruebas/Season_21-22.csv')
    
    teams_dict = {'Athletic Club': 'Athletic Club' ,
                 'CA Osasuna':'Osasuna',
                 'Club Atlético de Madrid':'Atlético Madrid',
                 'Cádiz CF':'Cádiz',
                 'Deportivo Alavés':'Alavés',
                 'Elche CF':'Elche',
                 'FC Barcelona':'Barcelona',
                 'Getafe CF':'Getafe',
                 'Granada CF':'Granada',
                 'Levante UD':'Levante',
                 'RC Celta de Vigo':'Celta Vigo',
                 'RCD Espanyol de Barcelona':'Espanyol',
                 'RCD Mallorca':'Mallorca',
                 'Rayo Vallecano de Madrid':'Rayo Vallecano',
                 'Real Betis Balompié':'Betis',
                 'Real Madrid CF':'Real Madrid',
                 'Real Sociedad de Fútbol':'Real Sociedad',
                 'Sevilla FC':'Sevilla',
                 'Valencia CF':'Valencia',
                 'Villarreal CF':'Villarreal'}
    
    new_team_name = []
    for team in comunio.Team:
  
        for k,v in teams_dict.items():
       
            if k == team:
           
                new_team_name.append(v)
    
    comunio['Team'] = new_team_name
    
    points_per_team = comunio.groupby('Team').sum().reset_index()
    points_per_team= points_per_team.rename(columns={'Total_Points': 'Squad_Points',
                                                       'Points_Average': 'Squad_Average_Points',
                                                       'Avg_last_5_games': 'Squad_Avg_last_5_Games',
                                                       'Value': 'Value_Squad',
                                                       f'J_{journey -4}': f'Squad_Points_J_{journey -4}',
                                                       f'J_{journey -3}': f'Squad_Points_J_{journey -3}',
                                                       f'J_{journey -2}': f'Squad_Points_J_{journey -2}',
                                                       f'J_{journey -1}': f'Squad_Points_J_{journey -1}',
                                                       f'J_{journey}': f'Squad_Points_J_{journey}',})
    
    points_per_team = points_per_team.drop(['Team_id','On_start_%'], axis=1)
    
    df_1 = comunio.merge(points_per_team, how='left', left_on='Team', right_on='Team')
    
    matches_J = cal.loc[cal['Journey'] == (journey+1)]
    matches_J
    
    vs = []
    for team in df_1.Team:

        for index in matches_J.index:
           
            if team == matches_J.loc[index].Home:
            
                vs.append(matches_J.loc[index].Away)
                
            if team == matches_J.loc[index].Away:
                
                vs.append(matches_J.loc[index].Home)
    
    df_1['vs'] = vs
    
    points_per_vs_team = comunio.groupby('Team').sum().reset_index()
    points_per_vs_team= points_per_vs_team.rename(columns={'Total_Points': 'Vs_Squad_Points',
                                                           'Team':'Vs_Team',
                                                           'Points_Average': 'Vs_Squad_Average_Points',
                                                           'Avg_last_5_games': 'Vs_Squad_Avg_last_5_Games',
                                                           'Value':'Vs_Value_Squad',
                                                           f'J_{journey -4}': f'Vs_Squad_Points_J_{journey -4}',
                                                           f'J_{journey -3}': f'Vs_Squad_Points_J_{journey -3}',
                                                           f'J_{journey -2}': f'Vs_Squad_Points_J_{journey -2}',
                                                           f'J_{journey -1}': f'Vs_Squad_Points_J_{journey -1}',
                                                           f'J_{journey}': f'Vs_Squad_Points_J_{journey}',})
    
    points_per_vs_team = points_per_vs_team.drop(['Team_id','On_start_%'], axis=1)
    
    df_2 = df_1.merge(points_per_vs_team, how='left', left_on='vs', right_on='Vs_Team')
    
    team_clas = []

    for team in comunio.Team:
        for index in clas.index:
            
            if team in clas.loc[index].Team:
                
                team_clas.append(clas.loc[index].Position)
    
    team_clas_vs = []

    for team in df_2.Vs_Team:
        for index in clas.index:
           
            if team in clas.loc[index].Team:
                
                team_clas_vs.append(clas.loc[index].Position)
    
    df_2['Team_clas'] = team_clas
    df_2['Vs_Team_clas'] = team_clas_vs
    
    
    return df_2.drop('Team_id', axis=1)

In [3]:
j_28 = create_df(28)
j_29 = create_df(29)

In [4]:
j_28

,Team,Player,Position,Total_Points,Points_Average,Value,On_start_%,Last_5_games_points,J_24,J_25,J_26,J_27,J_28,Avg_last_5_games,Squad_Points,Squad_Average_Points,Value_Squad,Squad_Points_J_24,Squad_Points_J_25,Squad_Points_J_26,Squad_Points_J_27,Squad_Points_J_28,Squad_Avg_last_5_Games,vs,Vs_Team,Vs_Squad_Points,Vs_Squad_Average_Points,Vs_Value_Squad,Vs_Squad_Points_J_24,Vs_Squad_Points_J_25,Vs_Squad_Points_J_26,Vs_Squad_Points_J_27,Vs_Squad_Points_J_28,Vs_Squad_Avg_last_5_Games,Team_clas,Vs_Team_clas
0,Athletic Club,Unai Simón,PT,113,4.7,3800000,86,"['1', '7', '-1', '2', '5']",1,7,-1,2,5,2.8,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15
1,Athletic Club,Agirrezabala,PT,20,5.0,470000,14,"['-', '-', '-', '-', '-']",0,0,0,0,0,0.0,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15
2,Athletic Club,Ezkieta,PT,0,0.0,200000,0,"['-', '-', '-', '-', '-']",0,0,0,0,0,0.0,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15
3,Athletic Club,Iñigo Martínez,DF,115,5.5,6580000,71,"['-', '6', '0', '7', '-']",0,6,0,7,0,2.6,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15
4,Athletic Club,Daniel Vivian,DF,92,5.8,3210000,54,"['5', '13', '2', '2', '4']",5,13,2,2,4,5.2,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15
5,Athletic Club,Lekue,DF,82,3.6,1740000,75,"['0', '-', '3', '-', '-']",0,0,3,0,0,0.6,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15
6,Athletic Club,De Marcos,DF,73,5.2,1940000,39,"['-', '8', '-', '3', '2']",0,8,0,3,2,2.6,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15
7,Athletic Club,Yeray Álvarez,DF,72,4.8,2970000,54,"['-', '-', '4', '5', '4']",0,0,4,5,4,2.6,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15
8,Athletic Club,Balenziaga,DF,71,3.1,1660000,75,"['1', '-', '-1', '-', '3']",1,0,-1,0,3,0.6,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15
9,Athletic Club,Yuri Berchiche,DF,24,4.8,4280000,11,"['8', '5', '-', '5', '4']",8,5,0,5,4,4.4,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15


In [5]:
j_29

,Team,Player,Position,Total_Points,Points_Average,Value,On_start_%,Last_5_games_points,J_25,J_26,J_27,J_28,J_29,Avg_last_5_games,Squad_Points,Squad_Average_Points,Value_Squad,Squad_Points_J_25,Squad_Points_J_26,Squad_Points_J_27,Squad_Points_J_28,Squad_Points_J_29,Squad_Avg_last_5_Games,vs,Vs_Team,Vs_Squad_Points,Vs_Squad_Average_Points,Vs_Value_Squad,Vs_Squad_Points_J_25,Vs_Squad_Points_J_26,Vs_Squad_Points_J_27,Vs_Squad_Points_J_28,Vs_Squad_Points_J_29,Vs_Squad_Avg_last_5_Games,Team_clas,Vs_Team_clas
0,Athletic Club,Unai Simón,PT,115,4.6,3650000,86,"['7', '-1', '2', '5', '2']",7,-1,2,5,2,3.0,1778,91.5,76820000,101,22,84,54,66,65.4,Elche,Elche,1490,84.0,43990000,61,24,53,71,34,48.6,8,14
1,Athletic Club,Agirrezabala,PT,20,5.0,460000,14,"['-', '-', '-', '-', '-']",0,0,0,0,0,0.0,1778,91.5,76820000,101,22,84,54,66,65.4,Elche,Elche,1490,84.0,43990000,61,24,53,71,34,48.6,8,14
2,Athletic Club,Ezkieta,PT,0,0.0,200000,0,"['-', '-', '-', '-', '-']",0,0,0,0,0,0.0,1778,91.5,76820000,101,22,84,54,66,65.4,Elche,Elche,1490,84.0,43990000,61,24,53,71,34,48.6,8,14
3,Athletic Club,Iñigo Martínez,DF,115,5.5,6110000,69,"['6', '0', '7', '-', '-']",6,0,7,0,0,2.6,1778,91.5,76820000,101,22,84,54,66,65.4,Elche,Elche,1490,84.0,43990000,61,24,53,71,34,48.6,8,14
4,Athletic Club,Daniel Vivian,DF,98,5.8,3550000,55,"['13', '2', '2', '4', '6']",13,2,2,4,6,5.4,1778,91.5,76820000,101,22,84,54,66,65.4,Elche,Elche,1490,84.0,43990000,61,24,53,71,34,48.6,8,14
5,Athletic Club,Lekue,DF,82,3.6,1550000,72,"['-', '3', '-', '-', '-']",0,3,0,0,0,0.6,1778,91.5,76820000,101,22,84,54,66,65.4,Elche,Elche,1490,84.0,43990000,61,24,53,71,34,48.6,8,14
6,Athletic Club,Yeray Álvarez,DF,76,4.8,2780000,55,"['-', '4', '5', '4', '4']",0,4,5,4,4,3.4,1778,91.5,76820000,101,22,84,54,66,65.4,Elche,Elche,1490,84.0,43990000,61,24,53,71,34,48.6,8,14
7,Athletic Club,De Marcos,DF,75,5.0,1970000,41,"['8', '-', '3', '2', '2']",8,0,3,2,2,3.0,1778,91.5,76820000,101,22,84,54,66,65.4,Elche,Elche,1490,84.0,43990000,61,24,53,71,34,48.6,8,14
8,Athletic Club,Balenziaga,DF,71,3.1,1500000,72,"['-', '-1', '-', '3', '-']",0,-1,0,3,0,0.4,1778,91.5,76820000,101,22,84,54,66,65.4,Elche,Elche,1490,84.0,43990000,61,24,53,71,34,48.6,8,14
9,Athletic Club,Yuri Berchiche,DF,37,6.2,4000000,14,"['5', '-', '5', '4', '13']",5,0,5,4,13,5.4,1778,91.5,76820000,101,22,84,54,66,65.4,Elche,Elche,1490,84.0,43990000,61,24,53,71,34,48.6,8,14


In [6]:
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

In [7]:
j_28_target = j_29[['Player',f'J_29']]
j_28_target= j_28_target.rename(columns={'Player': 'Jugador', f'J_29':'Target'})
df= j_28.merge(j_28_target, how='left', left_on='Player', right_on='Jugador')
df

,Team,Player,Position,Total_Points,Points_Average,Value,On_start_%,Last_5_games_points,J_24,J_25,J_26,J_27,J_28,Avg_last_5_games,Squad_Points,Squad_Average_Points,Value_Squad,Squad_Points_J_24,Squad_Points_J_25,Squad_Points_J_26,Squad_Points_J_27,Squad_Points_J_28,Squad_Avg_last_5_Games,vs,Vs_Team,Vs_Squad_Points,Vs_Squad_Average_Points,Vs_Value_Squad,Vs_Squad_Points_J_24,Vs_Squad_Points_J_25,Vs_Squad_Points_J_26,Vs_Squad_Points_J_27,Vs_Squad_Points_J_28,Vs_Squad_Avg_last_5_Games,Team_clas,Vs_Team_clas,Jugador,Target
0,Athletic Club,Unai Simón,PT,113,4.7,3800000,86,"['1', '7', '-1', '2', '5']",1,7,-1,2,5,2.8,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15,Unai Simón,2.0
1,Athletic Club,Agirrezabala,PT,20,5.0,470000,14,"['-', '-', '-', '-', '-']",0,0,0,0,0,0.0,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15,Agirrezabala,0.0
2,Athletic Club,Ezkieta,PT,0,0.0,200000,0,"['-', '-', '-', '-', '-']",0,0,0,0,0,0.0,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15,Ezkieta,0.0
3,Athletic Club,Iñigo Martínez,DF,115,5.5,6580000,71,"['-', '6', '0', '7', '-']",0,6,0,7,0,2.6,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15,Iñigo Martínez,0.0
4,Athletic Club,Daniel Vivian,DF,92,5.8,3210000,54,"['5', '13', '2', '2', '4']",5,13,2,2,4,5.2,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15,Daniel Vivian,6.0
5,Athletic Club,Lekue,DF,82,3.6,1740000,75,"['0', '-', '3', '-', '-']",0,0,3,0,0,0.6,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15,Lekue,0.0
6,Athletic Club,De Marcos,DF,73,5.2,1940000,39,"['-', '8', '-', '3', '2']",0,8,0,3,2,2.6,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15,De Marcos,2.0
7,Athletic Club,Yeray Álvarez,DF,72,4.8,2970000,54,"['-', '-', '4', '5', '4']",0,0,4,5,4,2.6,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15,Yeray Álvarez,4.0
8,Athletic Club,Balenziaga,DF,71,3.1,1660000,75,"['1', '-', '-1', '-', '3']",1,0,-1,0,3,0.6,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15,Balenziaga,0.0
9,Athletic Club,Yuri Berchiche,DF,24,4.8,4280000,11,"['8', '5', '-', '5', '4']",8,5,0,5,4,4.4,1712,90.5,78470000,53,101,22,84,54,62.8,Getafe,Getafe,1484,80.9,61290000,50,47,41,29,57,44.8,8,15,Yuri Berchiche,13.0


In [25]:
def preprocess_data(train,target,journey):
    j_28_target = target[['Player',f'J_{journey}']]
    j_28_target= j_28_target.rename(columns={'Player': 'Jugador', f'J_{journey}':'Target'})
    df= train.merge(j_28_target, how='left', left_on='Player', right_on='Jugador')
    df = df.dropna()
    X = df.drop(['Target'], axis=1)._get_numeric_data()
    y = df.Target
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
    
    X_train_s = x_scaler.fit_transform(X_train)
    X_test_s = x_scaler.transform(X_test)
    
    y_train_s = y_scaler.fit_transform(y_train.values.reshape(-1,1))
    y_test_s = y_scaler.transform(y_test.values.reshape(-1,1))
                    
    return X, y, X_train, X_test, y_train, y_test ,X_train_s, X_test_s, y_train_s, y_test_s

In [26]:
X, y, X_train, X_test,y_train,y_test ,X_train_s, X_test_s, y_train_s, y_test_s= preprocess_data(j_28,j_29,29)

In [27]:
X.shape, y.shape, X_train_s.shape, X_test_s.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape, y_train_s.shape, y_test_s.shape

((575, 30),
 (575,),
 (431, 30),
 (144, 30),
 (431, 30),
 (144, 30),
 (431,),
 (144,),
 (431, 1),
 (144, 1))

In [29]:
x_s = x_scaler.fit_transform(X_train)
x_ts = x_scaler.transform(X_test)

y_s = y_scaler.fit_transform(y_train.values.reshape(-1,1))
y_ts = y_scaler.fit(y_test.values.reshape(-1,1))

In [13]:
rfr = RandomForestRegressor()
dt = DecisionTreeRegressor()
xgbr = XGBRegressor()
gb = GradientBoostingRegressor()
mlpr = MLPRegressor(hidden_layer_sizes=([100,100,50]), activation='tanh' ,tol=1e-2,validation_fraction=0.15, early_stopping=True)
neural =  Sequential()

neural.add(Dense(500, activation='relu',input_dim=30))
neural.add(Dense(100, activation='relu'))
neural.add(Dense(50, activation='relu'))
neural.add(Dense(1, activation='softmax'))

neural.compile(loss="mean_squared_error",
             optimizer="adam")

models = (rfr, dt, xgbr, gb, mlpr)

2022-03-22 12:23:07.942013: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-22 12:23:07.942044: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-22 12:23:07.942058: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gobuub-ROG-Strix): /proc/driver/nvidia/version does not exist
2022-03-22 12:23:07.942300: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
for model in models:
    name = str(model)[:13]
    '''if 'Sequential' in name:
        
        model.fit(X_train_s,
                   y_train_s,
                   epochs=50,
                   validation_data=0.15)
        pred = model.predict(X_test_s)
        pred2 = y_scaler.inverse_transform(pred)
        mse = mean_squared_error(pred2,y_test, squared=False)
        r2 = r2_score(pred2, y_test)
        
        pd.DataFrame(model.history).plot(figsize=(8,5))
        plt.grid()
        plt.show()'''
        
        
    if 'MLP' in name:
        
        model.fit(X_train_s,y_train_s)
        model.score(X_train_s, y_train_s)
        pred = model.predict(X_test_s)
        pred_test = y_scaler.inverse_transform(pred.reshape(-1,1))
        mse = mean_squared_error(pred_test,y_test, squared=False)
        
        
    else:
        model.fit(X_train_s,y_train_s)
        model.score(X_train_s, y_train_s)
        pred = model.predict(X_test_s)
        pred2 = y_scaler.inverse_transform(pred.reshape(-1,1))
        mse = mean_squared_error(pred2,y_test, squared=False)
        r2 = r2_score(pred2, y_test)
    
    print (f'mse del modelo {name} = {mse}')
    print (f'R2 score del modelo {name} = {r2}')

/tmp/ipykernel_22469/2470809860.py:29: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_s,y_train_s)


mse del modelo RandomForestR = 2.980815860539601
R2 score del modelo RandomForestR = -1.9756546824301182
mse del modelo DecisionTreeR = 3.926406329796582
R2 score del modelo DecisionTreeR = -0.5897596562681824
mse del modelo XGBRegressor( = 3.1407010852605914
R2 score del modelo XGBRegressor( = -1.3119305075426593
mse del modelo GradientBoost = 3.133360860056258
R2 score del modelo GradientBoost = -1.453731995899732


/home/kike/anaconda3/envs/Entorno_Virtual_BT_Data_Science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kike/anaconda3/envs/Entorno_Virtual_BT_Data_Science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


mse del modelo MLPRegressor( = 2.9141895628556687
R2 score del modelo MLPRegressor( = -1.453731995899732


In [16]:
pred_xgb = xgbr.predict(x_scaler.transform(X))
pred_xgb2 = y_scaler.inverse_transform(pred_xgb.reshape(-1,1))
print('XGB MSE: ',mean_squared_error(pred_xgb2, y, squared=False))
pred_mlp = mlpr.predict(x_scaler.transform(X))
pred_mlp2 = y_scaler.inverse_transform(pred_mlp.reshape(-1,1))
print('MLP MSE: ',mean_squared_error(pred_mlp2, y, squared=False))

XGB MSE:  1.5719577022507116
MLP MSE:  2.7743129399867974


In [17]:
compare = pd.DataFrame()
compare['y'] = y
compare['xgb_pred'] = pred_xgb2
compare['mlp_pred'] = pred_mlp2
compare = round(compare)
compare['dif_xgb'] = compare.y == compare.xgb_pred 
compare['dif_mlp'] = compare.y == compare.mlp_pred
print('XGB: ', compare['dif_xgb'].value_counts())
print('MLP: ', compare['dif_mlp'].value_counts())
no_zeros = compare.loc[compare.y!=0]
no_zeros.dif_xgb.value_counts()
no_zeros.dif_mlp.value_counts()

XGB:  True     467
False    108
Name: dif_xgb, dtype: int64
MLP:  False    474
True     101
Name: dif_mlp, dtype: int64


False    253
True      46
Name: dif_mlp, dtype: int64

In [18]:
no_zeros.dif_xgb.value_counts()

True     228
False     71
Name: dif_xgb, dtype: int64

In [30]:
model = Sequential()
model.add(Dense(500, activation='relu',input_dim=30))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss="mean_squared_error",
             optimizer="adam")

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 500)               15500     
                                                                 
 dense_9 (Dense)             (None, 100)               50100     
                                                                 
 dense_10 (Dense)            (None, 50)                5050      
                                                                 
 dense_11 (Dense)            (None, 1)                 51        
                                                                 
Total params: 70,701
Trainable params: 70,701
Non-trainable params: 0
_________________________________________________________________


In [32]:
history = model.fit(x_s,
          y_s,
          epochs=50,
          validation_data=0.1)
pred = model.predict(x_ts)
pred2 = y_scaler.inverse_transform(pred)
mse = mean_squared_error(pred2,y_test, squared=False)
r2 = r2_score(pred2, y_test)
        

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid()
plt.show()

Epoch 1/50
 1/14 [=>............................] - ETA: 2s - loss: 0.2209

IndexError: list index out of range

In [ ]:
type(y_train_s)